In [251]:
#get data on ways fighter won/lost through dec, won/lost through sub, won/lost through ko/tko  
#tabulate outcomes in a table such as: Outcome, % chance, winner 
#FRONT END: give the user the ability to input two fighters (drop down menu)
# ---- FINISHED --- 
#extra: prevent users from picking two fighters that are from different weight classes
#--- LONG RUN --- 
# Improve model where it takes # of strikes (successful, attempted) , ground game/submissions duration, takedowns (successful, attempted)
# into consideration 

In [252]:
import pandas as pd 

In [253]:
df = pd.read_csv("UFC Matches April 2023.csv")

In [254]:
df.head()

,Full Name,Weight,Matchup,Outcome,Method,Round,Time
0,Tom Aaron,WEIGHT: 155 lbs.,NaN,NaN,NaN,NaN,NaN
1,Tom Aaron,WEIGHT: 155 lbs.,Tom Aaron\n \n Matt Ricehouse,LOSS,U-DEC,3.0,5:00
2,Tom Aaron,WEIGHT: 155 lbs.,Tom Aaron\n \n Eric Steenberg,WIN,SUB,1.0,0:56
3,Danny Abbadi,WEIGHT: 155 lbs.,NaN,NaN,NaN,NaN,NaN
4,Danny Abbadi,WEIGHT: 155 lbs.,Danny Abbadi\n \n Jorge Gurgel,LOSS,S-DEC,3.0,5:00


## Data Cleaning

We have to clean our dataset then group each fighter and get their distributions to determine their likelihood of winning/losing via decision, tko/ko or submission.

In [255]:
# delete first row of every fighter
nulls = df[["Matchup", "Outcome", 'Method', 'Round', 'Time']].isnull().all(axis=1) 
df = df.loc[~nulls, :]

#find matches that are upcoming and delete them
df = df[df["Outcome"] != 'NEXT']

# delete "weight text" in weight column
df['Weight'] = df['Weight'].str.replace('WEIGHT: ', '')

# keep only opponents name in matchup column
def remove_name(orig_matchup):
    last_index = orig_matchup.rfind('\n')
    new_string = orig_matchup[last_index +1:]
    return new_string.strip()

df['Matchup'] = df['Matchup'].apply(lambda x: remove_name(x))

#discovered there is a duplicate fighter name 
df.loc[[19227, 19228, 19229, 19230, 19231], 'Full Name'] = "Bruno Blindado Silva"

In [256]:
unique_methods = df['Method'].unique()

In [257]:
unique_methods

array(['U-DEC', 'SUB', 'S-DEC', 'KO/TKO', 'Overturned', 'Other', 'M-DEC',
       'CNC', 'DQ', 'SUB ', 'KO/TKO ', 'S-DEC ', 'Decision', 'U-DEC ',
       'M-DEC ', 'Overturned ', 'CNC '], dtype=object)

In [258]:
unique_outcomes = df['Outcome'].unique()

In [259]:
unique_outcomes

array(['LOSS', 'WIN', 'NC', 'DRAW'], dtype=object)

In [260]:
removed_outcomes = ['NC', 'DRAW']
removed_methods = ['Overturned', 'Other', 'CNC', 'DQ', 'Overturned ', 'CNC ']

In [261]:
filtered_df1 = df.loc[~df['Method'].isin(removed_methods)]
filtered_df2 = filtered_df1.loc[~df['Outcome'].isin(removed_outcomes)]
filtered_df2['Method'] = filtered_df2['Method'].apply(lambda x: x.strip())

<ipython-input-261-fcbd382802ee>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df2['Method'] = filtered_df2['Method'].apply(lambda x: x.strip())


In [262]:
df = filtered_df2

## Exploratory Data Analysis (EDA)

In [263]:
decisions = ['U-DEC', 'S-DEC', 'M-DEC', 'Decision']
kos = ['KO/TKO']
submissions = ['SUB']

In [264]:
decision_df = df[df['Method'].isin(decisions)]
decision_count = decision_df['Method'].count()

ko_df = df[df['Method'].isin(kos)]
ko_count = ko_df['Method'].count()

submission_df = df[df['Method'].isin(submissions)]
submission_count = submission_df['Method'].count()

print(decision_count/len(df))
print(ko_count/len(df))
print(submission_count/len(df))

0.428926871484206
0.34714409346603203
0.223929035049762


In [265]:
#make multipliers?
#0.4286023366508005 --> 43% end by decision
#0.34714409346603203 --> 35% end by ko
#0.223929035049762 --> 22% end by submission

In [266]:
#questions that could be of importance:
#what is the most common method of winning?
#for fighters that win through ko/submission what round does it typically occur in?
#is there a discrepancy in the way fighters win across different weight classes?

In [267]:
def transform_dataset(df):
    data = {
    'Total Fights': df.groupby(['Full Name'])['Matchup'].count(),
    'Total Wins': df.loc[df['Outcome'] == 'WIN'].groupby(['Full Name'])['Outcome'].count(),
    'Total Losses': df.loc[df['Outcome'] == 'LOSS'].groupby(['Full Name'])['Outcome'].count(),
    'Wins By Decision': df.loc[(df['Outcome'] == 'WIN') & (df['Method'].isin(decisions))].groupby(['Full Name'])['Outcome'].count(),
    'Wins By KO': df.loc[(df['Outcome'] == 'WIN') & (df['Method'].isin(kos))].groupby(['Full Name'])['Outcome'].count(),
    'Wins By Submission': df.loc[(df['Outcome'] == 'WIN') & (df['Method'].isin(submissions))].groupby(['Full Name'])['Outcome'].count(),
    'Loss By Decision': df.loc[(df['Outcome'] == 'LOSS') & (df['Method'].isin(decisions))].groupby(['Full Name'])['Outcome'].count(),
    'Loss By KO': df.loc[(df['Outcome'] == 'LOSS') & (df['Method'].isin(kos))].groupby(['Full Name'])['Outcome'].count(),
    'Loss By Submission': df.loc[(df['Outcome'] == 'LOSS') & (df['Method'].isin(submissions))].groupby(['Full Name'])['Outcome'].count()
}
    fighter_stats = pd.DataFrame(data)
    fighter_stats.fillna(0, inplace=True)
    return fighter_stats

In [268]:
df = transform_dataset(df)

In [269]:
df.head()

,Total Fights,Total Wins,Total Losses,Wins By Decision,Wins By KO,Wins By Submission,Loss By Decision,Loss By KO,Loss By Submission
AJ Dobson,3,1.0,2.0,0.0,0.0,1.0,2.0,0.0,0.0
AJ Fletcher,4,2.0,2.0,0.0,1.0,1.0,2.0,0.0,0.0
AJ Fonseca,1,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
AJ Matthews,1,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
Aalon Cruz,3,1.0,2.0,0.0,1.0,0.0,0.0,2.0,0.0


## Monte Carlo Simulation

In [270]:
import math
import random as rnd
import numpy.random as npr
import scipy.stats as ss

In [271]:
def calculate_p(statistic, total_fights):
    return statistic/total_fights

# def calculate_sd(statistic, total_fights):
#     mean = calculate_p(statistic, total_fights)
#     return math.sqrt((total_fights*mean)*(1-mean))

def calculate_sd(statistic, total_fights):
    p = calculate_p(statistic, total_fights)
    return math.sqrt((p*(1-p))/total_fights)

# def calculate_sd(statistic, total_fights):
#     p = calculate_p(statistic, total_fights)

#     if total_fights >= 15:
#         return math.sqrt((p*(1-p))/total_fights)
#     else:
#         return math.sqrt((total_fights*p)*(1-p))

In [272]:
def get_fighter_parameters(fighter1, fighter2):
    data = {
        
        "Decision Wins" : [df.loc[fighter1, "Wins By Decision"], df.loc[fighter2, "Wins By Decision"]],
        
        "KO Wins" : [df.loc[fighter1, "Wins By KO"], df.loc[fighter2, "Wins By KO"]],
        
        "Sub Wins" : [df.loc[fighter1, "Wins By Submission"], df.loc[fighter2, "Wins By Submission"]],
        
        "Decision Losses" : [df.loc[fighter1, "Loss By Decision"], df.loc[fighter2, "Loss By Decision"]],
        
        "KO Losses" : [df.loc[fighter1, "Loss By KO"], df.loc[fighter2, "Loss By KO"]],
        
        "Sub Losses" : [df.loc[fighter1, "Loss By Submission"], df.loc[fighter2, "Loss By Submission"]],
        
        "Decision Wins Prop" : [calculate_p(df.loc[fighter1, "Wins By Decision"], df.loc[fighter1, "Total Fights"]),
                                   calculate_p(df.loc[fighter2, "Wins By Decision"], df.loc[fighter2, "Total Fights"])], 
            
        "Decision Wins SD" : [calculate_sd(df.loc[fighter1, "Wins By Decision"], df.loc[fighter1, "Total Fights"]),
                                   calculate_sd(df.loc[fighter2, "Wins By Decision"], df.loc[fighter2, "Total Fights"])],
        
        "KO Wins Prop" : [calculate_p(df.loc[fighter1, "Wins By KO"], df.loc[fighter1, "Total Fights"]),
                                   calculate_p(df.loc[fighter2, "Wins By KO"], df.loc[fighter2, "Total Fights"])], 
            
        'KO Wins SD' : [calculate_sd(df.loc[fighter1, "Wins By KO"], df.loc[fighter1, "Total Fights"]),
                                   calculate_sd(df.loc[fighter2, "Wins By KO"], df.loc[fighter2, "Total Fights"])], 
           
        'Sub Wins Prop' : [calculate_p(df.loc[fighter1, "Wins By Submission"], df.loc[fighter1, "Total Fights"]),
                                   calculate_p(df.loc[fighter2, "Wins By Submission"], df.loc[fighter2, "Total Fights"])],
           
        'Sub Wins SD' : [calculate_sd(df.loc[fighter1, "Wins By Submission"], df.loc[fighter1, "Total Fights"]),
                                   calculate_sd(df.loc[fighter2, "Wins By Submission"], df.loc[fighter2, "Total Fights"])], 
            
        'Decision Loss Prop' : [calculate_p(df.loc[fighter1, "Loss By Decision"], df.loc[fighter1, "Total Fights"]),
                                   calculate_p(df.loc[fighter2, "Loss By Decision"], df.loc[fighter2, "Total Fights"])], 
            
        'Decision Loss SD' : [calculate_sd(df.loc[fighter1, "Loss By Decision"], df.loc[fighter1, "Total Fights"]),
                                   calculate_sd(df.loc[fighter2, "Loss By Decision"], df.loc[fighter2, "Total Fights"])], 
            
        'KO Loss Prop' : [calculate_p(df.loc[fighter1, "Loss By KO"], df.loc[fighter1, "Total Fights"]),
                                   calculate_p(df.loc[fighter2, "Loss By KO"], df.loc[fighter2, "Total Fights"])], 
            
        'KO Loss SD' : [calculate_sd(df.loc[fighter1, "Loss By KO"], df.loc[fighter1, "Total Fights"]),
                                   calculate_sd(df.loc[fighter2, "Loss By KO"], df.loc[fighter2, "Total Fights"])], 
           
        'Sub Loss Prop' : [calculate_p(df.loc[fighter1, "Loss By Submission"], df.loc[fighter1, "Total Fights"]),
                                   calculate_p(df.loc[fighter2, "Loss By Submission"], df.loc[fighter2, "Total Fights"])], 
            
        'Sub Loss SD' : [calculate_sd(df.loc[fighter1, "Loss By Submission"], df.loc[fighter1, "Total Fights"]),
                                   calculate_sd(df.loc[fighter2, "Loss By Submission"], df.loc[fighter2, "Total Fights"])],
        'Number of Matches' : [df.loc[fighter1, "Total Fights"], df.loc[fighter2, "Total Fights"]]
           
           }
    
    fighter_parameter_df = pd.DataFrame(data=data, index = [fighter1, fighter2])
    return fighter_parameter_df

In [273]:
# def gameSim(): binomial count
#     results = []
#     fighter1_dec_score = (rnd.gauss(matchup_df.iloc[0]['Decision Wins'],matchup_df.iloc[0]['Decision Wins SD'])+ 
#                               rnd.gauss(matchup_df.iloc[1]['Decision Losses'],matchup_df.iloc[1]['Decision Loss SD']))/2
#     fighter1_ko_score = (rnd.gauss(matchup_df.iloc[0]['KO Wins'],matchup_df.iloc[0]['KO Wins SD'])+ 
#                               rnd.gauss(matchup_df.iloc[1]['KO Losses'],matchup_df.iloc[1]['KO Loss SD']))/2
#     fighter1_sub_score = (rnd.gauss(matchup_df.iloc[0]['Sub Wins'],matchup_df.iloc[0]['Sub Wins SD'])+ 
#                               rnd.gauss(matchup_df.iloc[1]['Sub Losses'],matchup_df.iloc[1]['Sub Loss SD']))/2
#     fighter2_dec_score = (rnd.gauss(matchup_df.iloc[1]['Decision Wins'],matchup_df.iloc[1]['Decision Wins SD'])+ 
#                               rnd.gauss(matchup_df.iloc[0]['Decision Losses'],matchup_df.iloc[0]['Decision Loss SD']))/2
#     fighter2_ko_score = (rnd.gauss(matchup_df.iloc[1]['KO Wins'],matchup_df.iloc[1]['KO Wins SD'])+ 
#                               rnd.gauss(matchup_df.iloc[0]['KO Losses'],matchup_df.iloc[0]['KO Loss SD']))/2
#     fighter2_sub_score = (rnd.gauss(matchup_df.iloc[1]['Sub Wins'],matchup_df.iloc[1]['Sub Wins SD'])+ 
#                               rnd.gauss(matchup_df.iloc[0]['Sub Losses'],matchup_df.iloc[0]['Sub Loss SD']))/2   
    
#     results.append(fighter1_dec_score)
#     results.append(fighter1_ko_score)
#     results.append(fighter1_sub_score)    
#     results.append(fighter2_dec_score)    
#     results.append(fighter2_ko_score)    
#     results.append(fighter2_sub_score)
    
    
#     if max(results) == results[0]:
#         return "f1_dec"
#     elif max(results) == results[1]:
#         return "f1_ko"
#     elif max(results) == results[2]:
#         return "f1_sub"
#     elif max(results) == results[3]:
#         return "f2_dec"
#     elif max(results) == results[4]:
#         return "f2_ko"
#     elif max(results) == results[5]:
#         return "f2_sub"
#     else: return rnd.choice(["f1_dec", "f1_ko", "f1_sub", "f2_dec", "f2_ko", "f2_sub"])

In [274]:
# def gameSim(): binomial
#     results = []
#     fighter1_dec_score = (npr.binomial(matchup_df.iloc[0]["Number of Matches"], matchup_df.iloc[0]['Decision Wins Prop'])+ 
#                               npr.binomial(matchup_df.iloc[1]['Number of Matches'], matchup_df.iloc[1]['Decision Loss Prop']))/2
#     fighter1_ko_score = (npr.binomial(matchup_df.iloc[0]["Number of Matches"], matchup_df.iloc[0]['KO Wins Prop'])+ 
#                               npr.binomial(matchup_df.iloc[1]['Number of Matches'], matchup_df.iloc[1]['KO Loss Prop']))/2
#     fighter1_sub_score = (npr.binomial(matchup_df.iloc[0]["Number of Matches"], matchup_df.iloc[0]['Sub Wins Prop'])+ 
#                               npr.binomial(matchup_df.iloc[1]['Number of Matches'], matchup_df.iloc[1]['Sub Loss Prop']))/2
#     fighter2_dec_score = (npr.binomial(matchup_df.iloc[1]['Number of Matches'], matchup_df.iloc[1]['Decision Wins Prop'])+ 
#                               npr.binomial(matchup_df.iloc[0]['Number of Matches'], matchup_df.iloc[0]['Decision Loss Prop']))/2
#     fighter2_ko_score = (npr.binomial(matchup_df.iloc[1]['Number of Matches'], matchup_df.iloc[1]['KO Wins Prop'])+ 
#                               npr.binomial(matchup_df.iloc[0]['Number of Matches'], matchup_df.iloc[0]['KO Loss Prop']))/2
#     fighter2_sub_score = (npr.binomial(matchup_df.iloc[1]['Number of Matches'], matchup_df.iloc[1]['Sub Wins Prop'])+ 
#                               npr.binomial(matchup_df.iloc[0]['Number of Matches'], matchup_df.iloc[0]['Sub Loss Prop']))/2  
    
#     results.append(fighter1_dec_score)
#     results.append(fighter1_ko_score)
#     results.append(fighter1_sub_score)    
#     results.append(fighter2_dec_score)    
#     results.append(fighter2_ko_score)    
#     results.append(fighter2_sub_score)
    
    
#     if max(results) == results[0]:
#         return "f1_dec"
#     elif max(results) == results[1]:
#         return "f1_ko"
#     elif max(results) == results[2]:
#         return "f1_sub"
#     elif max(results) == results[3]:
#         return "f2_dec"
#     elif max(results) == results[4]:
#         return "f2_ko"
#     elif max(results) == results[5]:
#         return "f2_sub"
#     else: return rnd.choice(["f1_dec", "f1_ko", "f1_sub", "f2_dec", "f2_ko", "f2_sub"])

In [275]:
#binomial prop 
def gameSim(): 
    results = []
    fighter1_dec_score = (rnd.gauss(matchup_df.iloc[0]['Decision Wins Prop'],matchup_df.iloc[0]['Decision Wins SD'])+ 
                              rnd.gauss(matchup_df.iloc[1]['Decision Loss Prop'],matchup_df.iloc[1]['Decision Loss SD']))/2
    fighter1_ko_score = (rnd.gauss(matchup_df.iloc[0]['KO Wins Prop'],matchup_df.iloc[0]['KO Wins SD'])+ 
                              rnd.gauss(matchup_df.iloc[1]['KO Loss Prop'],matchup_df.iloc[1]['KO Loss SD']))/2
    fighter1_sub_score = (rnd.gauss(matchup_df.iloc[0]['Sub Wins Prop'],matchup_df.iloc[0]['Sub Wins SD'])+ 
                              rnd.gauss(matchup_df.iloc[1]['Sub Loss Prop'],matchup_df.iloc[1]['Sub Loss SD']))/2
    fighter2_dec_score = (rnd.gauss(matchup_df.iloc[1]['Decision Wins Prop'],matchup_df.iloc[1]['Decision Wins SD'])+ 
                              rnd.gauss(matchup_df.iloc[0]['Decision Loss Prop'],matchup_df.iloc[0]['Decision Loss SD']))/2
    fighter2_ko_score = (rnd.gauss(matchup_df.iloc[1]['KO Wins Prop'],matchup_df.iloc[1]['KO Wins SD'])+ 
                              rnd.gauss(matchup_df.iloc[0]['KO Loss Prop'],matchup_df.iloc[0]['KO Loss SD']))/2
    fighter2_sub_score = (rnd.gauss(matchup_df.iloc[1]['Sub Wins Prop'],matchup_df.iloc[1]['Sub Wins SD'])+ 
                              rnd.gauss(matchup_df.iloc[0]['Sub Loss Prop'],matchup_df.iloc[0]['Sub Loss SD']))/2   
    
    results.append(fighter1_dec_score)
    results.append(fighter1_ko_score)
    results.append(fighter1_sub_score)    
    results.append(fighter2_dec_score)    
    results.append(fighter2_ko_score)    
    results.append(fighter2_sub_score)
    
    
    if max(results) == results[0]:
        return "f1_dec"
    elif max(results) == results[1]:
        return "f1_ko"
    elif max(results) == results[2]:
        return "f1_sub"
    elif max(results) == results[3]:
        return "f2_dec"
    elif max(results) == results[4]:
        return "f2_ko"
    elif max(results) == results[5]:
        return "f2_sub"
    else: return rnd.choice(["f1_dec", "f1_ko", "f1_sub", "f2_dec", "f2_ko", "f2_sub"])

In [276]:
# mixed
# def gameSim(): 
#     results = []
    
#     if matchup_df.iloc[0]["Number of Matches"] >= 15 and matchup_df.iloc[1]["Number of Matches"] >= 15:
#         fighter1_dec_score = (rnd.gauss(matchup_df.iloc[0]['Decision Wins Prop'],matchup_df.iloc[0]['Decision Wins SD'])+ 
#                               rnd.gauss(matchup_df.iloc[1]['Decision Loss Prop'],matchup_df.iloc[1]['Decision Loss SD']))/2
#         fighter1_ko_score = (rnd.gauss(matchup_df.iloc[0]['KO Wins Prop'],matchup_df.iloc[0]['KO Wins SD'])+ 
#                               rnd.gauss(matchup_df.iloc[1]['KO Loss Prop'],matchup_df.iloc[1]['KO Loss SD']))/2
#         fighter1_sub_score = (rnd.gauss(matchup_df.iloc[0]['Sub Wins Prop'],matchup_df.iloc[0]['Sub Wins SD'])+ 
#                               rnd.gauss(matchup_df.iloc[1]['Sub Loss Prop'],matchup_df.iloc[1]['Sub Loss SD']))/2
#         fighter2_dec_score = (rnd.gauss(matchup_df.iloc[1]['Decision Wins Prop'],matchup_df.iloc[1]['Decision Wins SD'])+ 
#                               rnd.gauss(matchup_df.iloc[0]['Decision Loss Prop'],matchup_df.iloc[0]['Decision Loss SD']))/2
#         fighter2_ko_score = (rnd.gauss(matchup_df.iloc[1]['KO Wins Prop'],matchup_df.iloc[1]['KO Wins SD'])+ 
#                               rnd.gauss(matchup_df.iloc[0]['KO Loss Prop'],matchup_df.iloc[0]['KO Loss SD']))/2
#         fighter2_sub_score = (rnd.gauss(matchup_df.iloc[1]['Sub Wins Prop'],matchup_df.iloc[1]['Sub Wins SD'])+ 
#                               rnd.gauss(matchup_df.iloc[0]['Sub Loss Prop'],matchup_df.iloc[0]['Sub Loss SD']))/2  
        
#     elif matchup_df.iloc[0]["Number of Matches"] >= 15 and matchup_df.iloc[1]["Number of Matches"] < 15:
#         fighter1_dec_score = (rnd.gauss(matchup_df.iloc[0]['Decision Wins Prop'],matchup_df.iloc[0]['Decision Wins SD'])+
#                             npr.binomial(matchup_df.iloc[1]['Number of Matches'], matchup_df.iloc[1]['Decision Loss Prop']))/2
#         fighter1_ko_score = (rnd.gauss(matchup_df.iloc[0]['KO Wins Prop'],matchup_df.iloc[0]['KO Wins SD'])+
#                             npr.binomial(matchup_df.iloc[1]['Number of Matches'], matchup_df.iloc[1]['KO Loss Prop']))/2
#         fighter1_sub_score = (rnd.gauss(matchup_df.iloc[0]['Sub Wins Prop'],matchup_df.iloc[0]['Sub Wins SD'])+
#                             npr.binomial(matchup_df.iloc[1]['Number of Matches'], matchup_df.iloc[1]['Sub Loss Prop']))/2
#         fighter2_dec_score = (rnd.gauss(matchup_df.iloc[1]['Decision Wins Prop'],matchup_df.iloc[1]['Decision Wins SD'])+ 
#                             npr.binomial(matchup_df.iloc[0]['Number of Matches'], matchup_df.iloc[0]['Decision Loss Prop']))/2
#         fighter2_ko_score = (rnd.gauss(matchup_df.iloc[1]['KO Wins Prop'],matchup_df.iloc[1]['KO Wins SD'])+
#                             npr.binomial(matchup_df.iloc[0]['Number of Matches'], matchup_df.iloc[0]['KO Loss Prop']))/2
#         fighter2_sub_score = (rnd.gauss(matchup_df.iloc[1]['Sub Wins Prop'],matchup_df.iloc[1]['Sub Wins SD'])+
#                             npr.binomial(matchup_df.iloc[0]['Number of Matches'], matchup_df.iloc[0]['Sub Loss Prop']))/2 
        
#     elif matchup_df.iloc[0]["Number of Matches"] < 15 and matchup_df.iloc[1]["Number of Matches"] >= 15:
#         fighter1_dec_score = (npr.binomial(matchup_df.iloc[0]["Number of Matches"], matchup_df.iloc[0]['Decision Wins Prop'])+ 
#                             rnd.gauss(matchup_df.iloc[1]['Decision Loss Prop'],matchup_df.iloc[1]['Decision Loss SD']))/2
#         fighter1_ko_score = (npr.binomial(matchup_df.iloc[0]["Number of Matches"], matchup_df.iloc[0]['KO Wins Prop'])+
#                             rnd.gauss(matchup_df.iloc[1]['KO Loss Prop'],matchup_df.iloc[1]['KO Loss SD']))/2
#         fighter1_sub_score = (npr.binomial(matchup_df.iloc[0]["Number of Matches"], matchup_df.iloc[0]['Sub Wins Prop'])+
#                             rnd.gauss(matchup_df.iloc[1]['Sub Loss Prop'],matchup_df.iloc[1]['Sub Loss SD']))/2
#         fighter2_dec_score = (npr.binomial(matchup_df.iloc[1]['Number of Matches'], matchup_df.iloc[1]['Decision Wins Prop'])+
#                             rnd.gauss(matchup_df.iloc[0]['Decision Loss Prop'],matchup_df.iloc[0]['Decision Loss SD']))/2
#         fighter2_ko_score = (npr.binomial(matchup_df.iloc[1]['Number of Matches'], matchup_df.iloc[1]['KO Wins Prop'])+ 
#                             rnd.gauss(matchup_df.iloc[0]['KO Loss Prop'],matchup_df.iloc[0]['KO Loss SD']))/2
#         fighter2_sub_score = (npr.binomial(matchup_df.iloc[1]['Number of Matches'], matchup_df.iloc[1]['Sub Wins Prop'])+ 
#                             rnd.gauss(matchup_df.iloc[0]['Sub Loss Prop'],matchup_df.iloc[0]['Sub Loss SD']))/2
    
#     else: 
#         fighter1_dec_score = (npr.binomial(matchup_df.iloc[0]["Number of Matches"], matchup_df.iloc[0]['Decision Wins Prop'])+ 
#                               npr.binomial(matchup_df.iloc[1]['Number of Matches'], matchup_df.iloc[1]['Decision Loss Prop']))/2
#         fighter1_ko_score = (npr.binomial(matchup_df.iloc[0]["Number of Matches"], matchup_df.iloc[0]['KO Wins Prop'])+ 
#                               npr.binomial(matchup_df.iloc[1]['Number of Matches'], matchup_df.iloc[1]['KO Loss Prop']))/2
#         fighter1_sub_score = (npr.binomial(matchup_df.iloc[0]["Number of Matches"], matchup_df.iloc[0]['Sub Wins Prop'])+ 
#                               npr.binomial(matchup_df.iloc[1]['Number of Matches'], matchup_df.iloc[1]['Sub Loss Prop']))/2
#         fighter2_dec_score = (npr.binomial(matchup_df.iloc[1]['Number of Matches'], matchup_df.iloc[1]['Decision Wins Prop'])+ 
#                               npr.binomial(matchup_df.iloc[0]['Number of Matches'], matchup_df.iloc[0]['Decision Loss Prop']))/2
#         fighter2_ko_score = (npr.binomial(matchup_df.iloc[1]['Number of Matches'], matchup_df.iloc[1]['KO Wins Prop'])+ 
#                               npr.binomial(matchup_df.iloc[0]['Number of Matches'], matchup_df.iloc[0]['KO Loss Prop']))/2
#         fighter2_sub_score = (npr.binomial(matchup_df.iloc[1]['Number of Matches'], matchup_df.iloc[1]['Sub Wins Prop'])+ 
#                               npr.binomial(matchup_df.iloc[0]['Number of Matches'], matchup_df.iloc[0]['Sub Loss Prop']))/2 
    
#     results.append(fighter1_dec_score)
#     results.append(fighter1_ko_score)
#     results.append(fighter1_sub_score)    
#     results.append(fighter2_dec_score)    
#     results.append(fighter2_ko_score)    
#     results.append(fighter2_sub_score)
    
    
#     if max(results) == results[0]:
#         return "f1_dec"
#     elif max(results) == results[1]:
#         return "f1_ko"
#     elif max(results) == results[2]:
#         return "f1_sub"
#     elif max(results) == results[3]:
#         return "f2_dec"
#     elif max(results) == results[4]:
#         return "f2_ko"
#     elif max(results) == results[5]:
#         return "f2_sub"
#     else: return rnd.choice(["f1_dec", "f1_ko", "f1_sub", "f2_dec", "f2_ko", "f2_sub"])

In [277]:
def gamesSim(ns):
    matchesout = []
    fighter1_decwin = 0
    fighter1_kowin = 0
    fighter1_subwin = 0
    fighter2_decwin = 0
    fighter2_kowin = 0
    fighter2_subwin = 0
    tie = 0
    for i in range(ns):
        gm = gameSim()
        matchesout.append(gm)
        if gm == "f1_dec":
            fighter1_decwin +=1 
        elif gm == "f1_ko":
            fighter1_kowin +=1 
        elif gm == "f1_sub":
            fighter1_subwin +=1 
        elif gm == "f2_dec":
            fighter2_decwin +=1 
        elif gm == "f2_ko":
            fighter2_kowin +=1 
        else:
            fighter2_subwin +=1 

    
    print(matchup_df.index[0] +' Decision Win ', round((fighter1_decwin/ns)*100,3),'%')
    print(matchup_df.index[0] +' KO Win ', round((fighter1_kowin/ns)*100,3),'%')
    print(matchup_df.index[0] +' Submission Win ', round((fighter1_subwin/ns)*100,3),'%')
    print(matchup_df.index[1] +' Decision Win ', round((fighter2_decwin/ns)*100,3),'%')
    print(matchup_df.index[1] +' KO Win ', round((fighter2_kowin/ns)*100,3),'%')
    print(matchup_df.index[1] +' Submission Win ', round((fighter2_subwin/ns)*100,3),'%')
    #print('Tie ', (tie/ns)*100, '%')
    
    #return matchesout

## Predictions on UFC Fight Night: Pavlovich vs Blaydes

### Sergei Pavlovich vs Curtis Blaydes

In [278]:
matchup_df = get_fighter_parameters("Sergei Pavlovich", "Curtis Blaydes")
gamesSim(10000)
#sergei ko

#perfect

Sergei Pavlovich Decision Win  0.0 %
Sergei Pavlovich KO Win  93.03 %
Sergei Pavlovich Submission Win  0.0 %
Curtis Blaydes Decision Win  0.04 %
Curtis Blaydes KO Win  6.93 %
Curtis Blaydes Submission Win  0.0 %


### Brad Tavares vs Bruno Silva

In [279]:
matchup_df = get_fighter_parameters("Brad Tavares", "Bruno Blindado Silva")
gamesSim(10000)
#bruno silva ko

#------------------------------>= 15 swapped
#winner wrong, 2nd option correct

#------------------------------ binomial
#winner wrong, 2nd option correct

#----------------------------- binomial prop
#winner wrong, 2nd option correct

#----------------------------- binomial prop
#winner wrong, 2nd option correct

Brad Tavares Decision Win  54.34 %
Brad Tavares KO Win  0.0 %
Brad Tavares Submission Win  0.33 %
Bruno Blindado Silva Decision Win  0.0 %
Bruno Blindado Silva KO Win  45.33 %
Bruno Blindado Silva Submission Win  0.0 %


### Bobby Green vs Jared Gordon - No Contest

In [280]:
#matchup_df = get_fighter_parameters("Bobby Green", "Jared Gordon")
#gamesSim(10000)

### Iasmin Lucindo vs Brogan Walker

In [281]:
matchup_df = get_fighter_parameters("Iasmin Lucindo", "Brogan Walker")
gamesSim(10000)
#iasmin decision
#winner correct, method wrong

Iasmin Lucindo Decision Win  0.0 %
Iasmin Lucindo KO Win  100.0 %
Iasmin Lucindo Submission Win  0.0 %
Brogan Walker Decision Win  0.0 %
Brogan Walker KO Win  0.0 %
Brogan Walker Submission Win  0.0 %


### Jeremiah Wells vs Matthew Semelsberger

In [282]:
matchup_df = get_fighter_parameters("Jeremiah Wells", "Matthew Semelsberger")
gamesSim(10000)
#jeremiah decision

#------------------- >=15 swapped
#winner correct, method wrong

#--------------------------------- binomial
#winner wrong, 2nd option correct

#--------------------------------- binomial prop
#winner correct, method wrong

#----------------------------- binomial prop
#all wrong

Jeremiah Wells Decision Win  3.66 %
Jeremiah Wells KO Win  64.63 %
Jeremiah Wells Submission Win  12.24 %
Matthew Semelsberger Decision Win  15.62 %
Matthew Semelsberger KO Win  3.85 %
Matthew Semelsberger Submission Win  0.0 %


### Rick Glenn vs Christos Giagos

In [283]:
matchup_df = get_fighter_parameters("Ricky Glenn", "Christos Giagos")
gamesSim(10000)
#christos ko

#---------------------------- >= 15 swapped
#winner correct, method wrong

#---------------------------- >= binomial
#winner correct, method wrong

#--------------------------------- binomial
#winner correct, method wrong

#----------------------------- binomial prop
#winner correct, method wrong

Ricky Glenn Decision Win  38.04 %
Ricky Glenn KO Win  0.78 %
Ricky Glenn Submission Win  1.16 %
Christos Giagos Decision Win  59.88 %
Christos Giagos KO Win  0.0 %
Christos Giagos Submission Win  0.14 %


### Rani Yahya vs Montel Jackson

In [284]:
matchup_df = get_fighter_parameters("Rani Yahya", "Montel Jackson")
gamesSim(10000)
#montel ko 

#----------------------------->= 15 swapped 
#winner correct, method wrong

#----------------------------->= binomial 
#winner wrong, 2nd option correct

#--------------------------------- binomial prop
#winner correct, method wrong

#----------------------------- binomial prop
#winner wrong, 2nd option correct

Rani Yahya Decision Win  15.2 %
Rani Yahya KO Win  0.0 %
Rani Yahya Submission Win  8.46 %
Montel Jackson Decision Win  75.42 %
Montel Jackson KO Win  0.92 %
Montel Jackson Submission Win  0.0 %


### Karol Rosa vs Norma Dumont

In [285]:
matchup_df = get_fighter_parameters("Karol Rosa", "Norma Dumont")
gamesSim(10000)
#norma decision

#------------------------------->= 15 swapped
#winner wrong, 2nd option correct

#------------------------------- binomial
#winner wrong, 2nd option correct

#--------------------------------- binomial prop
#winner wrong, 2nd option correct

#----------------------------- binomial prop
#winner wrong, 2nd option correct

Karol Rosa Decision Win  68.61 %
Karol Rosa KO Win  0.0 %
Karol Rosa Submission Win  0.0 %
Norma Dumont Decision Win  31.39 %
Norma Dumont KO Win  0.0 %
Norma Dumont Submission Win  0.0 %


### Mohammed Usman vs Junior Tafa - N/A (Junior Tafa's first fight recorded on fightmetric)

### Francis Marshall vs William Gomis

In [286]:
matchup_df = get_fighter_parameters("Francis Marshall", "William Gomis")
gamesSim(10000)
#william decision
#perfect

#----------------- binomial
#all wrong

#--------------------------------- binomial prop
#perfect

#----------------------------- binomial prop
#all wrong

Francis Marshall Decision Win  7.54 %
Francis Marshall KO Win  7.81 %
Francis Marshall Submission Win  0.0 %
William Gomis Decision Win  84.65 %
William Gomis KO Win  0.0 %
William Gomis Submission Win  0.0 %


### Brady Hiestand vs Batgerel Danaa

In [287]:
matchup_df = get_fighter_parameters("Brady Hiestand", "Batgerel Danaa")
gamesSim(10000)
#brady ko

#------------------------------ >= 15 swapped
#winner correct, method wrong

#------------------------------ binomial
#winner correct, method wrong

#--------------------------------- binomial prop
#winner correct, method wrong

#----------------------------- binomial prop
#winner correct, method wrong

Brady Hiestand Decision Win  64.1 %
Brady Hiestand KO Win  0.44 %
Brady Hiestand Submission Win  0.0 %
Batgerel Danaa Decision Win  21.53 %
Batgerel Danaa KO Win  13.93 %
Batgerel Danaa Submission Win  0.0 %


In [288]:
#-------------------------------------- >= 15 swapped
#perfect: 2/9 
#2nd accepted: 4/9
#winner: 7/9 winner correct

#--------------------------------------- binomial 
#perfect: 1/9 
#2nd accepted: 5/9
#winner: 5/9

#-------------------------------------- binomial prop 
#perfect: 2/9 perfect 
#2nd accepted: 4/9 
#winner: 7/9 

## Predictions on UFC Fight Night: Song vs Simón

### Song Yadong vs Ricky Simón

In [289]:
matchup_df = get_fighter_parameters("Song Yadong", "Ricky Simon")
gamesSim(10000)
#song ko

#winner wrong, 2nd option correct 

Song Yadong Decision Win  13.65 %
Song Yadong KO Win  32.96 %
Song Yadong Submission Win  0.0 %
Ricky Simon Decision Win  50.31 %
Ricky Simon KO Win  0.68 %
Ricky Simon Submission Win  2.4 %


### Caio Borralho vs Michal Oleksiejczuk

In [290]:
matchup_df = get_fighter_parameters("Caio Borralho", "Michal Oleksiejczuk")
gamesSim(10000)
#caio submission

#----------------------------- >= 15 swapped
#winner correct, method wrong

Caio Borralho Decision Win  90.65 %
Caio Borralho KO Win  0.26 %
Caio Borralho Submission Win  0.08 %
Michal Oleksiejczuk Decision Win  0.0 %
Michal Oleksiejczuk KO Win  9.01 %
Michal Oleksiejczuk Submission Win  0.0 %


### Rodolfo Vieira vs Cody Brundage

In [291]:
matchup_df = get_fighter_parameters("Rodolfo Vieira", "Cody Brundage")
gamesSim(10000)
#rodolfo submission
#perfect

Rodolfo Vieira Decision Win  2.2 %
Rodolfo Vieira KO Win  17.81 %
Rodolfo Vieira Submission Win  55.43 %
Cody Brundage Decision Win  1.99 %
Cody Brundage KO Win  2.08 %
Cody Brundage Submission Win  20.49 %


### Julian Erosa vs Fernando Padilla - N/A (Fernando Padilla's first fight recorded on fightmetric)

### Marcos Rogério de Lima vs Waldo Cortes-Acosta

In [292]:
matchup_df = get_fighter_parameters("Marcos Rogerio de Lima", "Cody Brundage")
gamesSim(10000)
#marcos decision

#---------------------------- >=15 swapped
#winner correct, 3rd option correct

Marcos Rogerio de Lima Decision Win  5.42 %
Marcos Rogerio de Lima KO Win  69.3 %
Marcos Rogerio de Lima Submission Win  0.01 %
Cody Brundage Decision Win  0.02 %
Cody Brundage KO Win  1.4 %
Cody Brundage Submission Win  23.85 %


### Josh Quinlan vs Trey Waters

In [293]:
matchup_df = get_fighter_parameters("Josh Quinlan", "Trey Waters")
gamesSim(10000)
#trey decision
#all wrong 

Josh Quinlan Decision Win  0.0 %
Josh Quinlan KO Win  100.0 %
Josh Quinlan Submission Win  0.0 %
Trey Waters Decision Win  0.0 %
Trey Waters KO Win  0.0 %
Trey Waters Submission Win  0.0 %


### Martin Buday vs Jake Collier

In [294]:
matchup_df = get_fighter_parameters("Martin Buday", "Jake Collier")
gamesSim(10000)
#martin decision
#perfect

Martin Buday Decision Win  71.82 %
Martin Buday KO Win  27.81 %
Martin Buday Submission Win  0.0 %
Jake Collier Decision Win  0.37 %
Jake Collier KO Win  0.0 %
Jake Collier Submission Win  0.0 %


### Cody Durden vs Charles Johnson

In [295]:
matchup_df = get_fighter_parameters("Cody Durden", "Charles Johnson")
gamesSim(10000)
#cody decision
#perfect

Cody Durden Decision Win  85.57 %
Cody Durden KO Win  1.03 %
Cody Durden Submission Win  0.0 %
Charles Johnson Decision Win  2.49 %
Charles Johnson KO Win  2.86 %
Charles Johnson Submission Win  8.05 %


In [296]:
#3/7 perfect
#4/7 if 2nd highest outcome is accepted
#5/7 winner correct

### Stephanie Egger vs Irina Alekseeva - N/A (Irina Alekseeva first fight recorded on fightmetric)

### Journey Newson vs Marcus McGhee - N/A (Marcus McGhee first fight recorded on fightmetric)

### Hailey Cowan vs Jamey-Lyn Horth - N/A (Jamey-Lyn Horth first fight recorded on fightmetric)

## Predictions on UFC 288: Sterling vs. Cejudo

### Aljamain Sterling vs Henry Cejudo

In [297]:
matchup_df = get_fighter_parameters("Aljamain Sterling", "Henry Cejudo")
gamesSim(10000)
#aljamain decision

#------------------- >=15 swapped
#perfect

#------------------- >=20 swapped
#all wrong, 2nd option correct

#------------------- binomial
#perfect

#--------------------------------- binomial prop
#winner wrong, 2nd option correct


Aljamain Sterling Decision Win  27.42 %
Aljamain Sterling KO Win  0.26 %
Aljamain Sterling Submission Win  0.29 %
Henry Cejudo Decision Win  63.49 %
Henry Cejudo KO Win  8.54 %
Henry Cejudo Submission Win  0.0 %


### Belal Muhammad vs Gilbert Burns

In [298]:
matchup_df = get_fighter_parameters("Belal Muhammad", "Gilbert Burns")
gamesSim(10000)
#belal decision

#------------------- >=15 swapped
#perfect

#------------------- >=20 swapped
#all wrong, 2nd option correct

#------------------- binomial
#perfect

#--------------------------------- binomial prop
#perfect

Belal Muhammad Decision Win  93.73 %
Belal Muhammad KO Win  0.03 %
Belal Muhammad Submission Win  0.0 %
Gilbert Burns Decision Win  6.2 %
Gilbert Burns KO Win  0.01 %
Gilbert Burns Submission Win  0.03 %


### Jessica Andrade vs Yan Xiaonan

In [299]:
matchup_df = get_fighter_parameters("Jessica Andrade", "Yan Xiaonan")
gamesSim(10000)
#yan ko

#---------------------------- >= 15 swapped
#all wrong, method wrong

#---------------------------- >= 20 swapped
#all wrong, method wrong

#------------------------ binomial
#winner correct, method wrong

#--------------------------------- binomial prop
#winner correct, method wrong


Jessica Andrade Decision Win  1.33 %
Jessica Andrade KO Win  0.17 %
Jessica Andrade Submission Win  0.0 %
Yan Xiaonan Decision Win  98.5 %
Yan Xiaonan KO Win  0.0 %
Yan Xiaonan Submission Win  0.0 %


### Movsar Evloev vs Bryce Mitchell

In [300]:
matchup_df = get_fighter_parameters("Movsar Evloev", "Bryce Mitchell")
gamesSim(10000)
#movsar decision
#perfect

Movsar Evloev Decision Win  95.35 %
Movsar Evloev KO Win  0.0 %
Movsar Evloev Submission Win  0.0 %
Bryce Mitchell Decision Win  4.65 %
Bryce Mitchell KO Win  0.0 %
Bryce Mitchell Submission Win  0.0 %


### Kron Gracie vs Charles Jourdain

In [301]:
matchup_df = get_fighter_parameters("Kron Gracie", "Charles Jourdain")
gamesSim(10000)
#charles decision

#------------------>= 15 swapped
#perfect

#------------------>= 20 swapped
#all wrong, 2nd option correct

#------------------ binomial
#all wrong

#--------------------------------- binomial prop
#perfect
#summing percentages: winner wrong


Kron Gracie Decision Win  13.57 %
Kron Gracie KO Win  0.0 %
Kron Gracie Submission Win  43.03 %
Charles Jourdain Decision Win  42.25 %
Charles Jourdain KO Win  1.1 %
Charles Jourdain Submission Win  0.05 %


### Drew Dober vs Matt Frevola

In [302]:
matchup_df = get_fighter_parameters("Drew Dober", "Matt Frevola")
gamesSim(10000)
#matt ko

#------------------ >= 15 swapped
#all wrong, method wrong

#------------------ >= 20 swapped
#all wrong, method wrong 

#---------------------------- binomial
#all wrong

#--------------------------------- binomial prop
#winner wrong, method wrong


Drew Dober Decision Win  1.41 %
Drew Dober KO Win  82.99 %
Drew Dober Submission Win  0.0 %
Matt Frevola Decision Win  10.31 %
Matt Frevola KO Win  1.61 %
Matt Frevola Submission Win  3.68 %


### Kennedy Nzechukwu vs Devin Clark

In [303]:
matchup_df = get_fighter_parameters("Kennedy Nzechukwu", "Devin Clark")
gamesSim(10000)
#kennedy submission

#------------------------------ >= 15 swapped
#winner wrong, 3rd option correct

#------------------------------ >= 20 swapped
#winner correct, method wrong

#------------------------------ binomial
#winner correct, method wrong

#--------------------------------- binomial prop
#winner correct, method wrong


Kennedy Nzechukwu Decision Win  0.93 %
Kennedy Nzechukwu KO Win  69.86 %
Kennedy Nzechukwu Submission Win  0.13 %
Devin Clark Decision Win  29.04 %
Devin Clark KO Win  0.04 %
Devin Clark Submission Win  0.0 %


### Khaos Williams vs Rolando Bedoya - N/A (Rolando Bedoya first fight recorded on fightmetric)

### Marina Rodriguez vs Virna Jandiroba

In [304]:
matchup_df = get_fighter_parameters("Marina Rodriguez", "Virna Jandiroba")
gamesSim(10000)
#virna decision

#--------------------- >= 15 swapped
#all wrong, method wrong

#--------------------- >= 20 swapped
#all wrong, method wrong

#--------------------- binomial 
#winner wrong, 2nd option correct

#--------------------------------- binomial prop
#all wrong


Marina Rodriguez Decision Win  97.62 %
Marina Rodriguez KO Win  0.21 %
Marina Rodriguez Submission Win  0.0 %
Virna Jandiroba Decision Win  0.65 %
Virna Jandiroba KO Win  0.58 %
Virna Jandiroba Submission Win  0.94 %


### Braxton Smith vs Parker Porter - N/A (Braxton Smith first fight recorded on fightmetric)

### Phil Hawes vs Ikram Aliskerov

In [305]:
matchup_df = get_fighter_parameters("Phil Hawes", "Ikram Aliskerov")
gamesSim(10000)
#ikram ko

#---------------------------- >= 15 swapped
#winner correct, 3rd option correct

#---------------------------- >= 20 swapped
#winner correct, 2nd option correct

#---------------------------- binomial
#winner wrong, 2nd option correct

#--------------------------------- binomial prop
#winner correct, method wrong

Phil Hawes Decision Win  0.0 %
Phil Hawes KO Win  0.01 %
Phil Hawes Submission Win  0.0 %
Ikram Aliskerov Decision Win  0.0 %
Ikram Aliskerov KO Win  0.03 %
Ikram Aliskerov Submission Win  99.96 %


### Rafael Estevam vs Zhalgas Zhumagulov - Cancelled

### Joseph Holmes vs Claudio Ribeiro

In [306]:
matchup_df = get_fighter_parameters("Joseph Holmes", "Claudio Ribeiro")
gamesSim(10000)
#claudio ko

#--------------------- >=15 swapped
#perfect

#--------------------- >=20 swapped
#perfect

#--------------------- binomial
#all wrong

#--------------------------------- binomial prop
#winner wrong, 2nd option correct

Joseph Holmes Decision Win  0.0 %
Joseph Holmes KO Win  32.16 %
Joseph Holmes Submission Win  27.24 %
Claudio Ribeiro Decision Win  4.43 %
Claudio Ribeiro KO Win  31.93 %
Claudio Ribeiro Submission Win  4.24 %


### Daniel Santos vs Johnny Munoz - Cancelled

In [307]:
#------------------------------- binomial
#perfect: 3/10
#2nd accepted: 5/10
#winner: 5/10

#--------------------------------- binomial prop
#3/10 perfect
#5/10 if 2nd option is accepted
#5/10 winner correct (summing)

In [308]:
#try as n gets larger mean np is paired with sd sqrt(npq) and compare to current and to binomial

In [309]:
#https://courses.lumenlearning.com/introstats1/chapter/a-population-proportion/